# Example of prototyping an SDFormat file

## Imports

In [ ]:
import os
from tempfile import mkdtemp

from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    DiagramBuilder,
    Meshcat,
    MeshcatVisualizerCpp,
    Parser,
    Simulator,
)

## Model

In [ ]:
tmp_dir = mkdtemp()
model_file = os.path.join(tmp_dir, "test_model.sdf")

In [ ]:
# Note: This is not a comprehensive set of variables, but just shows
# simple f-strings in Python.
mass_0 =  100

model_text = f"""\
<?xml version="1.0"?>
<!-- Provided SDF model originally taken from
     https://bitbucket.org/osrf/sdformat/src/deca28cd6cd5/test/integration/model/double_pendulum.sdf
     has undergone modifications -->
<sdf version="1.8">
  <model name="double_pendulum_with_base">
    <link name="base">
      <inertial>
        <mass>{mass_0}</mass>
      </inertial>
      <visual name="vis_plate_on_ground">
        <pose degrees="true">0 0 0.01 0 0 0</pose>
        <geometry>
          <cylinder>
            <radius>0.8</radius>
            <length>0.02</length>
          </cylinder>
        </geometry>
      </visual>
      <visual name="vis_pole">
        <pose>-0.275 0 1.1 0 0 0</pose>
        <geometry>
          <box>
            <size>0.2 0.2 2.2</size>
          </box>
        </geometry>
      </visual>
      <collision name="col_plate_on_ground">
        <pose>0 0 0.01 0 0 0</pose>
        <geometry>
          <cylinder>
            <radius>0.8</radius>
            <length>0.02</length>
          </cylinder>
        </geometry>
      </collision>
      <collision name="col_pole">
        <pose>-0.275 0 1.1 0 0 0</pose>
        <geometry>
          <box>
            <size>0.2 0.2 2.2</size>
          </box>
        </geometry>
      </collision>
    </link>

    <!-- upper link, length 1, angle -90 degrees -->
    <link name="upper_link">
      <pose relative_to="base" degrees="true">0 0 2.1 -90 0 0</pose>
      <self_collide>0</self_collide>
      <inertial>
        <pose>0 0 0.5 0 0 0</pose>
        <mass>30.0</mass>
      </inertial>
      <visual name="vis_upper_joint">
        <pose degrees="true">-0.05 0 0 0 90 0</pose>
        <geometry>
          <cylinder>
            <radius>0.1</radius>
            <length>0.3</length>
          </cylinder>
        </geometry>
      </visual>
      <visual name="vis_lower_joint">
        <pose degrees="true">0 0 1.0 0 90 0</pose>
        <geometry>
          <cylinder>
            <radius>0.1</radius>
            <length>0.2</length>
          </cylinder>
        </geometry>
      </visual>
      <visual name="vis_cylinder">
        <pose>0 0 0.5 0 0 0</pose>
        <geometry>
          <cylinder>
            <radius>0.1</radius>
            <length>0.9</length>
          </cylinder>
        </geometry>
      </visual>
      <collision name="col_upper_joint">
        <pose degrees="true">-0.05 0 0 0 90 0</pose>
        <geometry>
          <cylinder>
            <radius>0.1</radius>
            <length>0.3</length>
          </cylinder>
        </geometry>
      </collision>
      <collision name="col_lower_joint">
        <pose degrees="true">0 0 1.0 0 90 0</pose>
        <geometry>
          <cylinder>
            <radius>0.1</radius>
            <length>0.2</length>
          </cylinder>
        </geometry>
      </collision>
      <collision name="col_cylinder">
        <pose>0 0 0.5 0 0 0</pose>
        <geometry>
          <cylinder>
            <radius>0.1</radius>
            <length>0.9</length>
          </cylinder>
        </geometry>
      </collision>
    </link>
    <!-- lower link, length 1, angle ~-120 degrees more -->
    <link name="lower_link">
      <pose relative_to="base" degrees="true">0.25 1.0 2.1 -120 0 0</pose>
      <self_collide>0</self_collide>
      <inertial>
        <pose>0 0 0.5 0 0 0</pose>
        <mass>30.0</mass>
      </inertial>
      <visual name="vis_lower_joint">
        <pose degrees="true">0 0 0 0 90 0</pose>
        <geometry>
          <cylinder>
            <radius>0.08</radius>
            <length>0.3</length>
          </cylinder>
        </geometry>
      </visual>
      <visual name="vis_cylinder">
        <pose>0 0 0.5 0 0 0</pose>
        <geometry>
          <cylinder>
            <radius>0.1</radius>
            <length>0.9</length>
          </cylinder>
        </geometry>
      </visual>
      <collision name="col_lower_joint">
        <pose degrees="true">0 0 0 0 90 0</pose>
        <geometry>
          <cylinder>
            <radius>0.08</radius>
            <length>0.3</length>
          </cylinder>
        </geometry>
      </collision>
      <collision name="col_cylinder">
        <pose>0 0 0.5 0 0 0</pose>
        <geometry>
          <cylinder>
            <radius>0.1</radius>
            <length>0.9</length>
          </cylinder>
        </geometry>
      </collision>
    </link>
    <!-- pin joint for upper link, at origin of upper link -->
    <joint name="upper_joint" type="revolute">
      <parent>base</parent>
      <child>upper_link</child>
      <axis>
        <xyz expressed_in="__model__">1 0 0</xyz>
        <limit>
          <effort>0.0</effort>
        </limit>
      </axis>
    </joint>
    <joint name="lower_joint" type="revolute">
      <parent>upper_link</parent>
      <child>lower_link</child>
      <axis>
        <xyz expressed_in="__model__">1 0 0</xyz>
        <limit>
          <effort>0.0</effort>
        </limit>
      </axis>
    </joint>
  </model>
</sdf>
"""

In [ ]:
# Write temporary file.
with open(model_file, "w") as f:
    f.write(model_text)

## Parse and Visualize

In [ ]:
meshcat = Meshcat()

In [ ]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)

# Add pendulum model.
model = Parser(plant, scene_graph).AddModelFromFile(model_file)
# - Weld base at origin.
base_link = plant.GetBodyByName("base", model)
plant.WeldFrames(plant.world_frame(), base_link.body_frame())

plant.Finalize()
visualizer = MeshcatVisualizerCpp.AddToBuilder(builder, scene_graph, meshcat)
diagram = builder.Build()

In [ ]:
simulator = Simulator(diagram)
simulator.set_target_realtime_rate(1.0)
simulator.AdvanceTo(5.0)